In [ ]:
import cv2
import os

In [ ]:
def detectFace(net,frame,confidence_threshold=0.7):
    frameOpencv = frame.copy()
    frameHeight = frameOpencv.shape[0]
    frameWidth = frameOpencv.shape[1]
    blob = cv2.dnn.blobFromImage(frameOpencv,1.0,(227,227),[124.96,115.97,106.13],swapRB=True,crop=False)
    net.setInput(blob)
    detections = net.forward()
    faceBoxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0,0,i,2]
        if confidence > confidence_threshold:
            x1 = int(detections[0,0,i,3]*frameWidth)
            y1 = int(detections[0,0,i,4]*frameHeight)
            x2 = int(detections[0,0,i,5]*frameWidth)
            y2 = int(detections[0,0,i,6]*frameHeight)
            faceBoxes.append([x1,y1,x2,y2])
            cv2.rectangle(frameOpencv,(x1,y1),(x2,y2),(0,255,0),int(round(frameHeight/150)),8)
    return frameOpencv,faceBoxes

faceProto='opencv_face_detector.pbtxt'
faceModel='opencv_face_detector_uint8.pb'
ageProto='age_deploy.prototxt'
ageModel='age_net.caffemodel'
genderProto='gender_deploy.prototxt'
genderModel='gender_net.caffemodel'

genderList=['Male','Female']
ageList=['(0-2)','(4-6)','(8-12)','(15-20)','(25-32)','(38-43)','(48-53)','(60-100)']

faceNet=cv2.dnn.readNet(faceModel,faceProto)
ageNet=cv2.dnn.readNet(ageModel,ageProto)
genderNet=cv2.dnn.readNet(genderModel,genderProto)

video=cv2.VideoCapture(0)
padding=20
while cv2.waitKey(1)<0:
    hasFrame, frame = video.read()
    if not hasFrame:
        cv2.waitKey()
        break

    resultImage,faceboxes = detectFace(faceNet,frame)

    if not faceboxes:
        print("No Face Detected")
        
    for facebox in faceboxes:
        face=frame[max(0,facebox[1]-padding):min(facebox[3]+padding,frame.shape[0]-1),max(0,facebox[0]-padding):min(facebox[2]+padding,frame.shape[1]-1)]
        blob = cv2.dnn.blobFromImage(face,1.0,(227,227),[124.96,115.97,106.13],swapRB=True,crop=False)
        genderNet.setInput(blob)
        genderPrediction = genderNet.forward()
        gender = genderList[genderPrediction[0].argmax()]
    
        ageNet.setInput(blob)
        agePrediction = ageNet.forward()
        age = ageList[agePrediction[0].argmax()]
        
        cv2.putText(resultImage,f'{gender},{age}',(facebox[0],facebox[1]-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,255,255),2,cv2.LINE_AA)
        cv2.imshow("Predicting the Age and Gender...",resultImage)
        
        if cv2.waitKey(33) & 0xFF == ord('q'):
            break

cv2.destroyAllWindows()

